In [1]:
# In[15]:
import sys

#######
DATA_IDX = 1
DATA_FNAME = '/oak/stanford/orgs/kipac/users/delon/LensQuEst/map_sims_%d.pkl'%(DATA_IDX)
print(DATA_FNAME)

N_RUNS = 10
import warnings
warnings.filterwarnings("ignore")
#####


# In[2]:


import os, sys
WORKING_DIR = os.path.dirname(os.path.abspath(''))
sys.path.insert(1, os.path.join(WORKING_DIR,'LensQuEst'))


# In[3]:


from universe import *
from halo_fit import *
from cmb import *
from flat_map import *
from weight import *
from pn_2d import *
import pickle
import seaborn as sns
from scipy.stats import spearmanr


# In[4]:


print("Map properties")

# number of pixels for the flat map
nX = 1200
nY = 1200

# map dimensions in degrees
sizeX = 20.
sizeY = 20.

# basic map object
baseMap = FlatMap(nX=nX, nY=nY, sizeX=sizeX*np.pi/180., sizeY=sizeY*np.pi/180.)

# multipoles to include in the lensing reconstruction
lMin = 30.; lMax = 3.5e3


from tqdm import trange,tqdm 
import pickle

from itertools import product

poss = list(product([True, False], range(N_RUNS)))


oup_fname = '../data/input/universe_Planck15/camb/CAMB_outputs.pkl'
print(oup_fname)
f = open(oup_fname, 'rb') 
powers,cl,c_lensed,c_lens_response = pickle.load(f)
f.close()
totCL=powers['total']
unlensedCL=powers['unlensed_scalar']

L = np.arange(unlensedCL.shape[0])

unlensedTT = unlensedCL[:,0]/(L*(L+1))*2*np.pi
F = unlensedTT
funlensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(cl.shape[0])
PP = cl[:,0]
rawPP = PP*2*np.pi/((L*(L+1))**2)
rawKK = L**4/4 * rawPP

fKK = interp1d(L, rawKK, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(totCL.shape[0])

lensedTT = totCL[:,0]/(L*(L+1))*2*np.pi
F = lensedTT
flensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)




ftot = lambda l : flensedTT(l) + cmb.fForeground(l) + cmb.fdetectorNoise(l)

L = np.arange(c_lens_response.shape[0])

cTgradT = c_lens_response.T[0]/(L*(L+1))*2*np.pi
fTgradT = interp1d(L, cTgradT, kind='linear', bounds_error=False, fill_value=0.)

# Adjust the lMin and lMax to the assumptions of the analysis
# CMB S4/SO specs
cmb = StageIVCMB(beam=1.4, noise=7., lMin=lMin, lMaxT=lMax, lMaxP=lMax, atm=False)

# Total power spectrum, for the lens reconstruction
# basiscally gets what we theoretically expect the
# power spectrum will look like
forCtotal = lambda l: ftot(l) 

# reinterpolate: gain factor 10 in speed
L = np.logspace(np.log10(lMin/2.), np.log10(2.*lMax), 1001, 10.)
F = np.array(list(map(forCtotal, L)))
cmb.fCtotal = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)


f = lambda l: np.sqrt(cmb.fCtotal(l))
clFourier = np.array(list(map(f, baseMap.l.flatten())))
clFourier = np.nan_to_num(clFourier)
clFourier = clFourier.reshape(np.shape(baseMap.l))


data = {}
frandomizePhase = lambda z: np.abs(z) * np.exp(1j*np.random.uniform(0., 2.*np.pi))

/oak/stanford/orgs/kipac/users/delon/LensQuEst/map_sims_1.pkl
Map properties
../data/input/universe_Planck15/camb/CAMB_outputs.pkl


In [2]:
import cProfile

In [ ]:
cProfile.run('baseMap.genGRF(ftot)')

In [9]:
f = lambda l: np.sqrt(ftot(l))
clFourier = np.array(list(map(f, baseMap.l.flatten())))
clFourier = np.nan_to_num(clFourier)
clFourier = clFourier.reshape(np.shape(baseMap.l))

In [10]:
cProfile.run('baseMap.genGRF(ftot, clFourier=clFourier)')

         66 function calls (60 primitive calls) in 0.092 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.003    0.003 <__array_function__ internals>:177(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(empty_like)
        1    0.000    0.000    0.009    0.009 <__array_function__ internals>:177(fft)
        1    0.000    0.000    0.009    0.009 <__array_function__ internals>:177(rfft)
        1    0.000    0.000    0.019    0.019 <__array_function__ internals>:177(rfftn)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(shape)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(swapaxes)
        1    0.000    0.000    0.003    0.003 <__array_function__ internals>:177(zeros_like)
        1    0.001    0.001    0.092    0.092 <string>:1(<module>)
        1    0.001    0.001    0.019    0.019 _poc